## Preamble

Import needed python and ROOT modules, load JPP/ROOT libraries

In [1]:
import ROOT, rootpy
from ROOT import gSystem
import root_numpy as rnp
import numpy as np
from dependencies import root_dependencies
root_dependencies()

DEBUG:rootpy.logger] Using rootpy's default log handler
DEBUG:rootpy.logger.magic] called SetErrorHandler()
DEBUG:rootpy.utils.quickroot] Loaded libHist (required by TH1)
DEBUG:rootpy] Using ROOT 5.34/36
DEBUG:rootpy.defaults] PyROOT's finalSetup() has been triggered
DEBUG:rootpy.defaults] PyROOT's finalSetup() has been called (gROOT.IsBatch()==False)
DEBUG:rootpy.defaults] configure_defaults()
DEBUG:rootpy.logger.magic] called SetErrorHandler()
/usr/local/lib/python2.7/dist-packages/root_numpy/__init__.py:35: RuntimeWarning: numpy 1.11.2 is currently installed but you installed root_numpy against numpy 1.11.1. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


In [ ]:
numuCC_files = !ls /home/cdesio/trigger_optimisation/neutrinos/numuCC/out_JTE/*.JTE.root

In [ ]:
fname = numuCC_files[30]

In [ ]:
fname

Get Montecarlo energies

In [ ]:
montecarlo_energies = rnp.root2array(filename, treename="MONTECARLO", branches="neutrino_.E_")

Get Montecarlo number of hits per event

In [ ]:
montecarlo_numberofhits = rnp.root2array(filename, treename="MONTECARLO", branches="@HitList_.size()")

In [ ]:
montecarlo_numberofhits = montecarlo_numberofhits.astype(np.int64)

In [ ]:
triggered_numberofhits = rnp.root2array(filename, treename="KM3NET_EVENT", branches="@triggeredHits.size()")

In [ ]:
triggered_numberofhits = triggered_numberofhits.astype(np.int64)

In [ ]:
sel_mc_hits = [i for i in montecarlo_numberofhits if i<5]

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Range1d
x = sel_energies
y = sel_mc_hits
output_notebook()
p = figure(title="Number of hits", x_axis_type="log",plot_height=400, plot_width=600)
#p.x_range = Range1d(1e2, 1e8)
#p.y_range = Range1d(1e-4, 1e4)
p.xaxis.axis_label = "Log Energy (GeV)"
p.yaxis.axis_label = "nhits per event"
r = p.cross(x, y, color="navy", line_width=1)
show(p)

In [ ]:
sel_energies = []
sel_hits = []
for i, en in enumerate(montecarlo_energies):
    if montecarlo_numberofhits[i] < 5:
        sel_energies.append(en)
        sel_hits.append(montecarlo_numberofhits[i])

In [ ]:
sel2 = np.array([en for i, en in enumerate(montecarlo_energies) if montecarlo_numberofhits[i]<5])

evaluate a possible efficiency as:
in each energy bin, the number of events with nhits>4 / total number of evetns in that bin.(combinel1 and nocomb)

In [ ]:
montecarlo_pmt_idk = rnp.root2array(filename, treename="MONTECARLO", branches="HitList_")


In [ ]:
montecarlo_pmt_idk

In [ ]:
montecarlo_pmt_id0 = rnp.root2array(filename, treename="MONTECARLO", branches="HitList_.pm_id_[1]")

In [ ]:
montecarlo_pmt_id1 = rnp.root2array(filename, treename="MONTECARLO", branches="HitList_.pm_id_[6000000]")

In [ ]:
montecarlo_pmt_id0

In [ ]:
from rootpy.io import root_open
tfile = root_open(filename)
tree = tfile.MONTECARLO

In [ ]:
64164/31

In [ ]:
montecarlo_pmt_id = montecarlo_pmt_id.astype(np.int64)

In [ ]:
binned = np.histogram(montecarlo_pmt_id, bins=31,range=(min(montecarlo_pmt_id), max(montecarlo_pmt_id)) )

```c++
root [6] tt->Scan("HitList_.pm_id_")
***********************************
*    Row   * Instance * HitList_. *
***********************************
*        0 *        0 *     45392 *
*        0 *        1 *     45983 *
*        0 *        2 *     45995 *
*        0 *        3 *     45996 *
*        0 *        4 *     46002 *
*        0 *        5 *     29217 *
*        0 *        6 *     29220 *
*        0 *        7 *     46000 *
*        0 *        8 *     47179 *
*        0 *        9 *     61091 *
*        0 *       10 *     44212 *
*        0 *       11 *     45989 *
*        0 *       12 *     46001 *
*        0 *       13 *     46001 *
*        0 *       14 *     60601 *
*        1 *        0 *     39923 *
*        2 *        0 *     53347 *
*        3 *        0 *     34291 *
*        3 *        1 *     34323 *
*        3 *        2 *     34402 *
*        3 *        3 *     35316 *
*        3 *        4 *     35319 *
*        3 *        5 *     35397 *
*        3 *        6 *     34390 *
*        3 *        7 *     36581 *
```

In [ ]:
from ROOT import TFile 
from root_numpy import tree2array
                     

f1=TFile(filename)
tree=f1.Get('MONTECARLO')

In [ ]:
eventnumber = rnp.root2array(filename, treename='MONTECARLO', branches='eventNumber_')
numberOfEvents = len(eventnumber)
max_evn = np.int(np.max(eventnumber))
ndoms = 2070

In [ ]:
print(numberOfEvents, max_evn)

```c++
 tt->Draw("HitList_.pm_id_/31:eventNumber_>> h(3853, 0, 6500,2070, 0, 2070)")
```

In [ ]:
del histo

In [ ]:
histo = ROOT.TH2F('dom hit per event','dom hit per event', numberOfEvents, 0, max_evn, ndoms, 0, ndoms)

In [ ]:
tree.Draw("HitList_.pm_id_/31:eventNumber_ >> histo","")

In [ ]:
tree.Draw("HitList_.pm_id_/31:eventNumber_>> h(numberOfEvents, 0, max_evn,ndoms, 0, ndoms)","")

In [ ]:
h.GetBinContent(20,34)

In [ ]:
histo.GetBin(100)

In [ ]:
bins = []
for x in range(numberOfEvents):
    for y in range(ndoms):
        bins.append((x, y, int(histo.GetBin(x,y))))

In [ ]:
ev_countdoms = []
for x in range(numberOfEvents):
    count = 0
    for y in range(ndoms):
        if int(histo.GetBinContent(x,y))!=0:
            count+=1
    if count!=0:
        ev_countdoms.append((x, count))
            

In [ ]:
ev_5doms = [ ev_countdoms[i][0] for i in range(len(ev_countdoms)) if ev_countdoms[i][1]>5]

In [ ]:
ev_countdoms

In [ ]:
histo.GetBinContent(0,1)

In [ ]:
histo.GetEffs()

https://root.cern.ch/root/roottalk/roottalk98/2318.html

### Using `rootpy.plotting`

In [ ]:
import rootpy.plotting

In [ ]:
from rootpy.io import root_open

In [ ]:
root_file = root_open(filename)

In [ ]:
rpy_tree = root_file.MONTECARLO

In [ ]:
rpy_tree

In [ ]:
h2d = rootpy.plotting.Hist2D(numberOfEvents, 0, max_evn, ndoms, 0, ndoms, type="F")

In [ ]:
ret = rpy_tree.Draw('eventNumber_:HitList_.pm_id_/31', hist=h2d, create_hist=False)

In [ ]:
bin_slice = h2d[0, :]
points = bin_slice.points
print(next(points))

In [ ]:
for i in range (2070):
    print(next(points))

In [ ]:
type(ret)

In [ ]:
h2d[0:]

In [ ]:
len(list(h2d.xedges()))

In [ ]:
max_evn / float(numberOfEvents)

In [ ]:
list(h2d.xedges())[:3]

In [ ]:
list(h2d.yedges())[:3]

In [ ]:
from rootpy.plotting import Hist2D

In [ ]:
h2d[2, :].effective_entries

In [ ]:
b0 = Hist2D(h2d[0, :])

In [ ]:
ev_countdoms = []
for x in range(numberOfEvents):
    count = 0
    for y in range(ndoms):
        if h2d.GetBinContent(x,y) > 0:
            count+=1
    if count != 0:
        ev_countdoms.append((x, count))

In [ ]:
ev_countdoms

In [ ]:
ev_countdoms = []
for x in range(4):
    count = 0
    for y in range(ndoms):
        if h2d.GetBinContent(x,y) > 0:
            count+=1
    if count != 0:
        ev_countdoms.append((x, count))

In [ ]:
ndoms*3


In [ ]:
ev_countdoms

In [ ]:
2070*3853

In [ ]:
h2d.GetSize()

In [ ]:
ev_countdoms

In [ ]:
ev_countdoms = []
for x in range(h2d.GetSize()/4):
    count = 0
    if h2d.GetBinContent(x,y) > 0:
        count+=1
    if count != 0:
        ev_countdoms.append((x, count))

In [ ]:
count=0
for x in range(3853*2, 3853*3):
        if h2d.GetBinContent(x) > 0:
            print(h2d.GetBinContent(x))
            count+=1
print(count)

In [ ]:
ev_countdoms

In [ ]:
for x in range(20):
    for y in range(20):
        print(x, y, h2d.GetBinContent(int(h2d.GetBin(x,y))))

In [ ]:
import pickle

In [ ]:
with open('/home/cdesio/notebooks/ev_arr.txt','rb') as f:
    ev_arr = pickle.load(f)

In [ ]:
len(ev_arr)

In [ ]:
import km3pipe as kp

pump = kp.GenericPump(fname)
geo = kp.Geometry(filename="/home/cdesio/trigger_optimisation/km3net_jul13_90m_r1494_corrected.detx")

for blob in pump:
    hits = blob['Hits']
    hit_doms = set(geo.detector.doms[h.dom_id][:2] for h in hits)
    hit_doms2 = set(geo.detector.doms[h.dom_id] for h in hits)
    #print(hit_doms)
    #break

In [ ]:
for f, d, p, in hit_doms2:
    print(f, d, p)

In [ ]:
import km3pipe as kp

pump = kp.GenericPump(fname)
doms = []
for blob in pump:
    #print(blob['Hits'].dom_id)
    #break
    np.array(doms.append((blob['Hits'].id, blob['Hits'].dom_id)))

In [ ]:
len(doms)

In [ ]:
triggered_evt = rnp.root2array(fname, treename='KM3NET_EVENT', branches='getCounter()')

In [ ]:
len(triggered_evt)

### looking at the number of doms per triggered event

In [28]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Range1d
TOOLS="resize,crosshair,pan,wheel_zoom,box_zoom,reset,tap,previewsave,box_select,poly_select,lasso_select"

In [2]:
fname = "../trigger_optimisation_files/km3_v4_numucc_1.evt_st_100ns_4hits.JTE.root"

In [3]:
ndoms = rnp.root2array(fname, treename="KM3NET_EVENT",branches="triggeredHits.getModuleID()")

In [4]:
ndoms.size

1858

In [5]:
print(ndoms)

[ array([1397, 1397, 1398, 1398, 1398, 1398, 1398, 1398, 1398, 1398, 1398,
       1399, 1399, 1399, 1399, 1399, 1401, 1401, 1401, 1401, 1907, 1907,
       1907], dtype=int32)
 array([1603, 1603, 1604, 1604, 1604, 1605, 1605, 1605, 1605, 1605, 1605,
       1606, 1606, 1606, 1607, 1607, 1607], dtype=int32)
 array([170, 170, 170, 170, 170, 170, 170, 170, 170, 171, 171, 171, 332,
       332, 332, 332], dtype=int32)
 ...,
 array([ 973,  973, 1495, 1495, 1513, 1513, 1513, 1513, 1513, 1513, 1514,
       1514, 1963, 1963, 1963, 1963, 1963, 1963, 1981, 1981, 1981, 1981,
       1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981,
       1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981,
       1981, 1981, 1981, 1982, 1982, 1982, 1982, 1982, 1982, 1983, 1983], dtype=int32)
 array([ 635,  635, 1027, ..., 2070, 2070, 2070], dtype=int32)
 array([1225, 1225, 1225, 1225, 1225, 1225, 1261, 1261, 1261, 1261, 1261,
       1261, 1261, 1261, 1261, 1261, 1261, 1261, 1261, 1261, 1

In [6]:
fnamest = "../trigger_optimisation_files/km3_v4_numucc_1.evt.JTE.root"

In [7]:
ndoms_st = rnp.root2array(fnamest, treename="KM3NET_EVENT",branches="triggeredHits.getModuleID()")

In [8]:
print(ndoms_st)

[ array([1397, 1397, 1398, 1398, 1398, 1398, 1398, 1398, 1398, 1398, 1398,
       1399, 1399, 1399, 1399, 1399, 1401, 1401, 1401, 1401, 1907, 1907,
       1907], dtype=int32)
 array([1603, 1603, 1604, 1604, 1604, 1605, 1605, 1605, 1605, 1605, 1606,
       1606, 1607, 1607], dtype=int32)
 array([ 392,  392,  393,  393,  393,  675,  675,  675,  675,  675,  675,
        675,  676,  676,  676,  676,  676,  676,  676, 1123, 1123, 1642,
       1642, 1642, 1643, 1643], dtype=int32)
 ...,
 array([ 973,  973, 1513, 1513, 1513, 1513, 1514, 1514, 1963, 1963, 1963,
       1963, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981,
       1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981,
       1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1982, 1982, 1983,
       1983], dtype=int32)
 array([1027, 1027, 1029, ..., 2070, 2070, 2070], dtype=int32)
 array([1225, 1225, 1225, 1225, 1225, 1225, 1261, 1261, 1261, 1261, 1261,
       1261, 1261, 1263, 1263, 1263, 1263, 1263, 1783, 17

In [19]:
ndoms[0]-ndoms_st[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [33]:
for i,j in zip(ndoms_st[:10], ndoms[:10]):
    print(i,j)
    

(array([1397, 1397, 1398, 1398, 1398, 1398, 1398, 1398, 1398, 1398, 1398,
       1399, 1399, 1399, 1399, 1399, 1401, 1401, 1401, 1401, 1907, 1907,
       1907], dtype=int32), array([1397, 1397, 1398, 1398, 1398, 1398, 1398, 1398, 1398, 1398, 1398,
       1399, 1399, 1399, 1399, 1399, 1401, 1401, 1401, 1401, 1907, 1907,
       1907], dtype=int32))
(array([1603, 1603, 1604, 1604, 1604, 1605, 1605, 1605, 1605, 1605, 1606,
       1606, 1607, 1607], dtype=int32), array([1603, 1603, 1604, 1604, 1604, 1605, 1605, 1605, 1605, 1605, 1605,
       1606, 1606, 1606, 1607, 1607, 1607], dtype=int32))
(array([ 392,  392,  393,  393,  393,  675,  675,  675,  675,  675,  675,
        675,  676,  676,  676,  676,  676,  676,  676, 1123, 1123, 1642,
       1642, 1642, 1643, 1643], dtype=int32), array([170, 170, 170, 170, 170, 170, 170, 170, 170, 171, 171, 171, 332,
       332, 332, 332], dtype=int32))
(array([ 818,  818,  818,  818,  818,  818,  818,  818,  818,  818,  818,
        818,  818,  818,  818,

In [23]:
new_st = np.zeros(ndoms.size)

In [25]:
import pickle

In [26]:
with open("moduleID_st", 'wb') as f:
    pickle.dump(ndoms_st, f)
with open("moduleID_100ns4h", "wb") as f:
    pickle.dump(ndoms, f)

In [29]:
import numpy as np
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_notebook

p1 = figure(title="moduleID", tools=TOOLS)
hist, edges = np.histogram(ndoms, density=False, bins=100)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")

p1.xaxis.axis_label=''

show(p1)

ValueError: operands could not be broadcast together with shapes (23,) (17,) 

In [47]:
ndoms_uniq=np.zeros(ndoms.size)
for i in ndoms:
    for j in ndoms[i]:
        if (j+1)!=j:
            ndoms_uniq[i]=j

IndexError: index 1907 is out of bounds for axis 0 with size 1858

In [48]:
module_counter = {}
for dom in ndoms:
    module_counter.setdefault(dom,0)
    module_counter[dom] += 1


TypeError: unhashable type: 'numpy.ndarray'

In [49]:
import sets

In [63]:
for evt1, evt2 in zip(ndoms, ndoms_st):
    print(len(evt1), sets.Set(evt1), len(evt2),sets.Set(evt2))

(23, Set([1401, 1907, 1397, 1398, 1399]), 23, Set([1401, 1907, 1397, 1398, 1399]))
(17, Set([1603, 1604, 1605, 1606, 1607]), 14, Set([1603, 1604, 1605, 1606, 1607]))
(16, Set([170, 171, 332]), 26, Set([675, 676, 392, 393, 1642, 1643, 1123]))
(24, Set([432, 537, 1418, 179, 933]), 33, Set([1285, 1266, 1263, 1265, 818, 819]))
(14, Set([858, 859]), 17, Set([1152, 1114, 1115, 1654]))
(19, Set([328, 310, 327]), 21, Set([1090, 1611, 749, 785, 786, 2042]))
(23, Set([1022, 589, 534, 1021]), 13, Set([1199, 419, 349, 350, 694]))
(19, Set([1968, 1969]), 19, Set([1794, 1795, 1796, 807]))
(15, Set([1400, 1907, 1399, 1908, 911]), 24, Set([194, 3, 196, 45, 46, 195, 91]))
(10, Set([385, 386, 1122, 349]), 11, Set([362, 1298, 398, 1837]))
(27, Set([352, 369, 370, 388]), 10, Set([673, 1162, 1163, 1237]))
(28, Set([2017, 2018]), 19, Set([784, 379, 380, 381, 783]))
(28, Set([675, 676, 392, 393, 1642, 1643, 1123]), 33, Set([656, 1589, 1069, 422, 423]))
(14, Set([290, 291, 292]), 19, Set([1536, 1537, 1985, 15

In [64]:
for evt1, evt2 in zip(ndoms, ndoms_st):
    if (sets.Set(evt1)==sets.Set(evt2)):
        print(sets.Set(evt1))
    elif (sets.Set(evt2) in sets.Set(evt1)):
        print sets.Set(evt1)

Set([1401, 1907, 1397, 1398, 1399])
Set([1603, 1604, 1605, 1606, 1607])


In [52]:
uniqevt

Set([1804, 1225, 1802, 1803, 1801, 1261, 1263, 1783, 1784])